### Geocoding API

In [1]:
import requests
from urllib.parse import urlencode
from dotenv import load_dotenv
import os
from urllib.parse import urlparse, parse_qsl
from pymongo import MongoClient
import json


In [2]:
load_dotenv()

True

In [3]:
api_google_maps = os.getenv("api_google_maps")

##### Geocode Api de Compañía de Madrid

In [4]:
data_type = "json"
enpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
params = {"address": "Calle Quintanavides 15, Madrid, Comunidad de Madrid", "key": api_google_maps}
url_params = urlencode(params)

new_url = f"{enpoint}?{url_params}"


In [5]:
def extract_lat_lng(address_or_postalcode, data_type = "json"):
    enpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    params = {"address": address_or_postalcode, "key": api_google_maps}
    url_params = urlencode(params)              
    new_url = f"{enpoint}?{url_params}"
    r = requests.get(new_url)
    if r.status_code not in range(200, 299):
        return {}
    latlng = {}
    try:
        latlng = r.json()["results"][0]["geometry"]["location"]
    except:
        pass
    return latlng.get("lat"), latlng.get("lng")

In [6]:
extract_lat_lng("Calle Quintanavides 15, Madrid, Comunidad de Madrid")

(40.4977863, -3.6697489)

In [7]:
to_parse = f"https://maps.googleapis.com/maps/api/geocode/json?address=Calle+Quintanavides+15%2C+Madrid%2C+Comunidad+de+Madrid&key={api_google_maps}"

In [37]:
#urlparse(to_parse)

In [38]:
parsed_url = urlparse(to_parse)
query_string = parsed_url.query


In [39]:
query_tuple = parse_qsl(query_string)


In [40]:
query_dict = dict(query_tuple)


In [12]:
endpoint = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
print(endpoint)

https://maps.googleapis.com/maps/api/geocode/json


###  Places API

In [13]:
# Vamos a hacer places solo 
# Con el lat y lng de las empresas que hemos escogido que nos dan las coordenadas de las ciudades que queremos
# la respuesta de data la convertiremos en json como hace el americano
# luego de vul

##### Geocode Api de Compañía de Madrid la más cerca

In [41]:
lat, lng = (40.4977863, -3.6697489)
base_enpoint_places = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
params = {
    "key": api_google_maps,
    "input": "Tech Companies",
    "inputtype": "textquery",
    "fields": "formatted_address,name,geometry,permanently_closed,business_status"
}
locationbias = f"circle:{lat},{lng}"
use_circular = True
if use_circular:
    radius = 400
    locationbias = f"circle:{radius}@{lat},{lng}"

params["locationbias"] = locationbias
    
params_encoded = urlencode(params)
places_endpoint = f"{base_enpoint_places}?{params_encoded}"


In [15]:
r = requests.get(places_endpoint)
print(r.status_code)

200


In [16]:
r.json()

{'candidates': [{'business_status': 'OPERATIONAL',
   'formatted_address': 'P.º de la Castellana, 182, 6ª Planta, 28046 Madrid, España',
   'geometry': {'location': {'lat': 40.4609226, 'lng': -3.6890381},
    'viewport': {'northeast': {'lat': 40.46228722989272,
      'lng': -3.687812420107278},
     'southwest': {'lat': 40.45958757010728, 'lng': -3.690512079892722}}},
   'name': 'Trappit'}],
 'status': 'OK'}

### Nearby Places API Radius

###### Madrid Company

Tech Companies near Madrid's company found through coordinates from "companies" dataframe in Mongo.

In [17]:
techs_endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_2 = {
    "key": api_google_maps,
    "location": f"{lat},{lng}",
    "radius": 500,
    "keyword": "Tech Companies"
}
params_2_encoded = urlencode(params_2)
techs_url = f"{techs_endpoint}?{params_2_encoded}"

r2 = requests.get(techs_url)

In [18]:
r2.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 40.5027322, 'lng': -3.6718831},
    'viewport': {'northeast': {'lat': 40.50414192989273,
      'lng': -3.670619770107277},
     'southwest': {'lat': 40.50144227010728, 'lng': -3.673319429892721}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Verbio Technologies- Madrid Office',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 512,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107204109335243508929">A Google User</a>'],
     'photo_reference': 'Aap_uECxyKPUeiJL2X5B74EJVsoSOucTPqFdLBE-ma8hZkQZWekuBwIh6mlPgCYwCbe5FWoMZtoKCqWJCzvSspay8nho6ZnM19jVdGr1jJyy4brN7MLeny9SGgWIZrKUMjoMG0RYLW6t_jBmhO4MLz_oUp41jI7yLJ8T_VGmudngOO5gq8tb',
     'width': 

Transport information near given company Madrid location

In [19]:
transport_endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_3 = {
    "key": api_google_maps,
    "location": f"{lat},{lng}",
    "radius": 300,
    "keyword": "Public Transport"
}
params_3_encoded = urlencode(params_3)
trans_url = f"{transport_endpoint}?{params_3_encoded}"

r3 = requests.get(trans_url)

In [20]:
r3.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 40.4968012, 'lng': -3.667288},
    'viewport': {'northeast': {'lat': 40.49812832989272,
      'lng': -3.665973570107278},
     'southwest': {'lat': 40.49542867010727, 'lng': -3.668673229892721}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Quintanavides - Av. de Burgos',
   'place_id': 'ChIJdc0Odf4rQg0RYv2b2VVfmR4',
   'plus_code': {'compound_code': 'F8WM+P3 Madrid',
    'global_code': '8CGRF8WM+P3'},
   'rating': 0,
   'reference': 'ChIJdc0Odf4rQg0RYv2b2VVfmR4',
   'scope': 'GOOGLE',
   'types': ['transit_station', 'point_of_interest', 'establishment'],
   'user_ratings_total': 0,
   'vicinity': 'Madrid'},
  {'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 

Bar/Cafeteria near given location

In [21]:
bar_endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_4 = {
    "key": api_google_maps,
    "location": f"{lat},{lng}",
    "radius": 600,
    "keyword": "bar"
}
params_4_encoded = urlencode(params_4)
bar_url = f"{bar_endpoint}?{params_4_encoded}"

r4 = requests.get(bar_url)

In [22]:
r4.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 40.4956666, 'lng': -3.6755234},
    'viewport': {'northeast': {'lat': 40.49699322989272,
      'lng': -3.674275820107279},
     'southwest': {'lat': 40.49429357010728, 'lng': -3.676975479892723}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/bar_pinlet',
   'name': 'Restaurante El Rincón de Las Tablas',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2268,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117141449752184430094">Enrique López Callejas</a>'],
     'photo_reference': 'Aap_uEDeYK_LG8I0Vc8txHu8L1CP8CyFSQGtSujrGz_m9_5nK29SWceibU8q6C0C5CTPz6HXMm2Wr6Sb_qYuIfS6RcEsm_n3DU2ZD8dR-xMqzTG1EYyvC7PrbyXufKuTkYBSNse61b7OfWmpFc5NX3R0ZcIPO09s9Smpek-eeerMOe4ugiTn',
     'width': 4032}]

###### London Company

In [23]:
latlnd, lngldn = (51.543337, -0.172364)

Near tach companies in London 1000m radius

In [24]:
techs_endpoint_ldn = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_5 = {
    "key": api_google_maps,
    "location": f"{latlnd},{lngldn}",
    "radius": 1000,
    "keyword": "Tech Companies"
}
params_5_encoded = urlencode(params_5)
techs_url_ldn = f"{techs_endpoint_ldn}?{params_5_encoded}"

r5 = requests.get(techs_url_ldn)

In [25]:
r5.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 51.5473213, 'lng': -0.1892146},
    'viewport': {'northeast': {'lat': 51.54862297989272,
      'lng': -0.1878524701072778},
     'southwest': {'lat': 51.54592332010727, 'lng': -0.1905521298927222}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Serac Technologies',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109387118224083587957">A Google User</a>'],
     'photo_reference': 'Aap_uEASmkrChd0TV_L4t2cjGUT1fmWa40rgs8aknr7is3yEJDxIbuf1hhpvcZLCDYSVjdl5zyNn3FEFD0zpgyYJqfemgRa3MhmVAY_K0lwhRKhUku9M1vj0wgXgYAdVX3hXwuwFQIzzoFXPVBMc9kZLCDI-5OjLejwH9ySgNtUuxpbAufEu',
     'width': 4032}],
   'p

Near Public Transport Services in a 400m radius

In [26]:
trans_endpoint_ldn = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_6 = {
    "key": api_google_maps,
    "location": f"{latlnd},{lngldn}",
    "radius": 400,
    "keyword": "Public Transport"
}
params_6_encoded = urlencode(params_6)
trans_url_ldn = f"{trans_endpoint_ldn}?{params_6_encoded}"

r6 = requests.get(trans_url_ldn)

In [27]:
r6.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 51.5469174, 'lng': -0.179986},
    'viewport': {'northeast': {'lat': 51.54822172989273,
      'lng': -0.1785647701072778},
     'southwest': {'lat': 51.54552207010728, 'lng': -0.1812644298927222}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Finchley Road tube station',
   'photos': [{'height': 2322,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115348551340047492204">Alex Marks</a>'],
     'photo_reference': 'Aap_uEDDSePWCFFyinI303tiQQJ7Q943i7l-ur3xYTL_BDRhFC7dbMjIHF1eP9CYVlrcBc4LPOjnIAuID7e5lLnKBpr6SkEOxud_4TDMYsxAijsfmCuCcUFfDOc9NLpOkDSzOLm2o2zjInLIhyyZiTrzlQceIkXVlwBZCSYxSjZ5-in8WXMM',
     'width': 4128}],
   'place_id': 'ChIJE_1U0IQadkgRc9Z_mCfWo

Near Bar/Cafeteria locals in a 600m radius

In [28]:
bar_endpoint_ldn = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_7 = {
    "key": api_google_maps,
    "location": f"{latlnd},{lngldn}",
    "radius": 600,
    "keyword": "bar"
}
params_7_encoded = urlencode(params_7)
bar_url_ldn = f"{bar_endpoint_ldn}?{params_7_encoded}"

r7 = requests.get(bar_url_ldn)

In [29]:
r7.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 51.541903, 'lng': -0.180838},
    'viewport': {'northeast': {'lat': 51.54330152989273,
      'lng': -0.1794057201072778},
     'southwest': {'lat': 51.54060187010729, 'lng': -0.1821053798927222}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/bar_pinlet',
   'name': 'The Arches',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 412,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111601843711282724568">NAILast Canary Wharf</a>'],
     'photo_reference': 'Aap_uEBZdXLT0gmPthKOWLpBvijYKmCLIVXkob-k2b_PCfIvyCUNbwyec7Om1gzEa-L_g4EhuF9fyPCWAieVfjLoy-zCW3y5U74dYqxwDVhiBmSv1IkuUnGxl4YPNh4Wp_8a4X8TvK_p9QVYY3AuEmQc2zNHffZKF3yqisuen2X9hbgLZUIZ',
     'width': 550}],
   'place_id': 'ChIJL7f2wp

###### Berlin Company

In [30]:
latber, lngber = (53.5771514, 10.0115926)

Near Tech companies in a 500m radius

In [31]:
techs_endpoint_ber = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_8 = {
    "key": api_google_maps,
    "location": f"{latber},{lngber}",
    "radius": 500,
    "keyword": "Tech Companies"
}
params_8_encoded = urlencode(params_8)
techs_url_ber = f"{techs_endpoint_ber}?{params_8_encoded}"

r8 = requests.get(techs_url_ber)

In [32]:
r8.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 53.5814629, 'lng': 10.0183172},
    'viewport': {'northeast': {'lat': 53.58269557989272,
      'lng': 10.01961352989272},
     'southwest': {'lat': 53.57999592010727, 'lng': 10.01691387010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Ventiv Technology',
   'place_id': 'ChIJYdB6jyuJsUcRZ5gWKUMtzoM',
   'plus_code': {'compound_code': 'H2J9+H8 Hamburg, Germany',
    'global_code': '9F5GH2J9+H8'},
   'rating': 0,
   'reference': 'ChIJYdB6jyuJsUcRZ5gWKUMtzoM',
   'scope': 'GOOGLE',
   'types': ['point_of_interest', 'establishment'],
   'user_ratings_total': 0,
   'vicinity': 'Hans-Henny-Jahnn-Weg 53, Hamburg'},
  {'business_status': 'OPERATIONAL',
   'geometry': {'location': {'la

Public Transport Services in a 400m radius

In [33]:
trans_endpoint_ber = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_9 = {
    "key": api_google_maps,
    "location": f"{latber},{lngber}",
    "radius": 400,
    "keyword": "Public Transport"
}
params_9_encoded = urlencode(params_9)
trans_url_ber = f"{techs_endpoint_ber}?{params_9_encoded}"

r9 = requests.get(trans_url_ber)

In [34]:
r9.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 53.5786806, 'lng': 10.013709},
    'viewport': {'northeast': {'lat': 53.58004692989272,
      'lng': 10.01512092989272},
     'southwest': {'lat': 53.57734727010728, 'lng': 10.01242127010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Hans-Henny-Jahnn-Weg',
   'place_id': 'ChIJaz9Q0dSOsUcRtW88a65_07o',
   'plus_code': {'compound_code': 'H2H7+FF Hamburg, Germany',
    'global_code': '9F5GH2H7+FF'},
   'rating': 0,
   'reference': 'ChIJaz9Q0dSOsUcRtW88a65_07o',
   'scope': 'GOOGLE',
   'types': ['transit_station', 'point_of_interest', 'establishment'],
   'user_ratings_total': 0,
   'vicinity': 'Hamburg'},
  {'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 

Bar/Cafeteria in a 400m radius

In [35]:
bar_endpoint_ber = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_10 = {
    "key": api_google_maps,
    "location": f"{latber},{lngber}",
    "radius": 400,
    "keyword": "bar"
}
params_10_encoded = urlencode(params_10)
bar_url_ber = f"{bar_endpoint_ber}?{params_10_encoded}"

r10 = requests.get(bar_url_ber)

In [36]:
r10.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 53.58049399999999, 'lng': 10.0125706},
    'viewport': {'northeast': {'lat': 53.58182172989272,
      'lng': 10.01382462989272},
     'southwest': {'lat': 53.57912207010727, 'lng': 10.01112497010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/bar_pinlet',
   'name': 'Zaza Bar',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 992,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106965784176101225832">A Google User</a>'],
     'photo_reference': 'Aap_uEBQzT6lmpgxAtnuIV-4cvtMhlvOXh2ga-252BFItUFlDSmMz079X8JJUVgQ2oksRnM7QQTtwxzwaZPOPqUKSVUEJwE7wIUvlhJtMwS8xwPygTsfef36zFpG2tdsSQTAuxp81eNpmwOF3xLUbARRky_rBPPw6RMwTcL_xQWvM8CYwY-n',
     'width': 1354}],
   'place_id': 'ChIJu12-kiqJs